In [1]:
library(Matrix)
library(dplyr)
library(rjson)
library(Seurat)
library(ggplot2)
#library(SeuratDisk)
library(Matrix)
#library(SpaceX)
library(cowplot)
library(patchwork)
library(tidyr)
# co-expression network analysis packages:
library(WGCNA)
library(hdWGCNA)
#library(SeuratData)
library(igraph)
library(Matrix)
library(enrichR)
enableWGCNAThreads(nThreads = 48)

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

websiteLive <- getOption("enrichR.live")
if (websiteLive) {
    listEnrichrSites()
    setEnrichrSite("Enrichr") # Human genes   
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching SeuratObject

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode


Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack


Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The

Allowing parallel execution with up to 48 working processes.


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



In [2]:
dbs <- c('GO_Biological_Process_2023','GO_Cellular_Component_2023','GO_Molecular_Function_2023')

In [196]:
table = read.csv("../Result/DE_genes_t_test/L1/severe_t_test.csv")
table = subset(table, pvalues_adj<0.05 & log2fc>0.5)

In [197]:
table

,X,scores,pvalues,pvalues_adj,log2fc,genes,pct,pct_rest
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,22883,161.45661,0,0,70.125605,MT-RNR2,1.0000000,1.00000000
2,22903,159.29342,0,0,15.884229,MT-TV,0.9944909,0.84503149
3,22882,158.30215,0,0,21.964151,MT-RNR1,1.0000000,0.99957764
4,22881,154.41292,0,0,7.454989,MT-ND6,0.9581765,0.60409307
5,21893,144.47748,0,0,10.155023,MALAT1,0.9994032,0.94582245
6,22893,126.22158,0,0,4.443551,MT-TL1,0.8722340,0.47304562
7,22871,115.04405,0,0,16.410458,MT-CO1,1.0000000,0.99938566
8,22869,113.34088,0,0,33.243168,MT-ATP6,1.0000000,0.99992321
9,22928,111.30023,0,0,5.415333,MTATP6P1,0.9809935,0.89502381


In [183]:
# Convert to lowercase
genes_lower <- tolower(gene_list)

# Capitalize the first letter
genes_proper <- sub("^(.)", "\\U\\1", genes_lower, perl = TRUE)

In [184]:
dbs <- c("GO_Molecular_Function_2023", "GO_Cellular_Component_2023", "GO_Biological_Process_2023")
if (websiteLive) {
    enriched <- enrichr(genes_proper, dbs)
}

Uploading data to Enrichr... Done.
  Querying GO_Molecular_Function_2023... Done.
  Querying GO_Cellular_Component_2023... Done.
  Querying GO_Biological_Process_2023... Done.
Parsing results... Done.


In [185]:
GO = subset(enriched[[3]], Adjusted.P.value < 0.05)

In [186]:
if (websiteLive) {
    plotEnrich(GO, showTerms = 40, numChar = 40, y = "Count", orderBy = "P.value")
}

ERROR: Error in plotEnrich(GO, showTerms = 40, numChar = 40, y = "Count", orderBy = "P.value"): Input df is empty.


In [187]:
table

,X,scores,pvalues,pvalues_adj,log2fc,genes,pct,pct_rest
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,21893,11.143245,4.917798e-18,2.169565e-17,11.942354,MALAT1,1.0000000,0.96202560
2,22881,10.726199,3.216619e-17,1.370117e-16,10.235270,MT-ND6,1.0000000,0.84963559
3,22882,10.685874,3.757211e-17,1.595812e-16,19.806592,MT-RNR1,1.0000000,0.99997778
4,22903,9.364839,1.507650e-14,5.653771e-14,15.214613,MT-TV,1.0000000,0.94327171
5,22883,8.536840,6.425716e-13,2.222557e-12,50.939164,MT-RNR2,1.0000000,1.00000000
6,23512,7.718182,2.679241e-11,8.464305e-11,3.920655,NEAT1,0.8536585,0.42680651
7,22902,7.521246,6.504335e-11,2.010484e-10,2.747108,MT-TT,0.7926829,0.39709804
8,22893,7.476899,7.958735e-11,2.451841e-10,3.851791,MT-TL1,0.8658537,0.63038841
9,4513,7.119743,3.949117e-10,1.167096e-09,3.622882,AC090204.1,0.6463415,0.24575593
